# Dealing with marks: the turmite model

The [turmite model](https://en.wikipedia.org/wiki/Langton's_ant), developed by [Christopher Langton](https://en.wikipedia.org/wiki/Christopher_Langton) in 1986, is a very simple mono-agent model that exhibits an emergent behavior. It is based on 2 rules:

* If the turmite is on a patch that does not contain a mark, it turns right, drops a mark, and moves forward,

* If the turmite is on a patch that contains a mark, it turns left, removes the mark, and moves forward.

## Grab dependencies

In [1]:
#!python
#@repository("~/.m2/repository/")
#@dependency(group="fr.univ-artois.lgi2a", module="similar2logo-lib", version="1.0-SNAPSHOT")

import math

from fr.univ_artois.lgi2a.similar.extendedkernel.libs.abstractimpl import AbstractAgtDecisionModel
from fr.univ_artois.lgi2a.similar.microkernel import AgentCategory
from fr.univ_artois.lgi2a.similar.microkernel.ISimulationModel import AgentInitializationData
from fr.univ_artois.lgi2a.similar2logo.kernel.initializations import AbstractLogoSimulationModel
from fr.univ_artois.lgi2a.similar2logo.kernel.model import LogoSimulationParameters
from fr.univ_artois.lgi2a.similar2logo.kernel.model.agents.turtle import TurtleFactory, \
    TurtleAgentCategory
from fr.univ_artois.lgi2a.similar2logo.kernel.model.environment import Mark, LogoEnvPLS
from fr.univ_artois.lgi2a.similar2logo.kernel.model.influences import ChangeDirection, \
    DropMark, RemoveMark
from fr.univ_artois.lgi2a.similar2logo.kernel.model.levels import LogoSimulationLevelList
from fr.univ_artois.lgi2a.similar2logo.lib.model import ConeBasedPerceptionModel
from fr.univ_artois.lgi2a.similar.microkernel.libs.probes import RealTimeMatcherProbe
from fr.univ_artois.lgi2a.similar2logo.lib.tools.web import Similar2LogoWebRunner

Resolving dependency: fr.univ-artois.lgi2a#similar2logo-lib;1.0-SNAPSHOT {default=[default]}
Preparing to download artifact fr.univ-artois.lgi2a#similar2logo-lib;1.0-SNAPSHOT!similar2logo-lib.jar
Preparing to download artifact fr.univ-artois.lgi2a#similar2logo-kernel;1.0-SNAPSHOT!similar2logo-kernel.jar
Preparing to download artifact fr.univ-artois.lgi2a#similar-microKernel;1.0-SNAPSHOT!similar-microKernel.jar
Preparing to download artifact fr.univ-artois.lgi2a#similar-microKernel-commonLibs;1.0-SNAPSHOT!similar-microKernel-commonLibs.jar
Preparing to download artifact fr.univ-artois.lgi2a#similar-extendedKernel;1.0-SNAPSHOT!similar-extendedKernel.jar
Preparing to download artifact fr.univ-artois.lgi2a#similar-extendedKernel-extendedLibs;1.0-SNAPSHOT!similar-extendedKernel-extendedLibs.jar
Preparing to download artifact org.json#json;20170516!json.jar(bundle)
Preparing to download artifact net.jafama#jafama;2.3.1!jafama.jar
Preparing to download artifact org.apache.commons#commons-math

No Outputs

## The decision model

The decision model implements the above described rules :

In [2]:
class TurmiteDecisionModel(AbstractAgtDecisionModel):
    
    def __init__(self):
        super(TurmiteDecisionModel, self).__init__(LogoSimulationLevelList.LOGO)
    
    def decide(self, timeLowerBound, timeUpperBound, globalState, publicLocalState, privateLocalState, perceivedData, producedInfluences):
        if perceivedData.marks.isEmpty():
            producedInfluences.add(
                ChangeDirection(
                    timeLowerBound,
                    timeUpperBound,
                    math.pi/2,
                    publicLocalState
                )
            )
            producedInfluences.add(
                DropMark(
                    timeLowerBound,
                    timeUpperBound,
                    Mark(
                        publicLocalState.location.clone(),
                        None
                    )
                )
            )
        else:
            producedInfluences.add(
                ChangeDirection(
                    timeLowerBound,
                    timeUpperBound,
                    -math.pi/2,
                    publicLocalState
                )
            )
            
            producedInfluences.add(
                RemoveMark(
                    timeLowerBound,
                    timeUpperBound,
                    perceivedData.marks.iterator().next().content
                )
            )

No Outputs

## The simulation model

The simulation model generates a turmite heading north at the location 10.5,10.5 with a speed of 1 and an acceleration of 0:

In [3]:
class TurmiteSimulationModel(AbstractLogoSimulationModel):
    
    def __init__(self, parameters):
        super(TurmiteSimulationModel, self).__init__(parameters)
    
    def generateAgents(self, parameters, levels):
        result = AgentInitializationData()
        turtle = TurtleFactory.generate(
            ConeBasedPerceptionModel(0.0, 2 * math.pi, False, True, False),
            TurmiteDecisionModel(),
            AgentCategory('turmite', [TurtleAgentCategory.CATEGORY]),
            LogoEnvPLS.NORTH,
            1.0,
            0.0,
            10.5,
            10.5
        )
        result.agents.add(turtle)
        return result

No Outputs

## Launch the HTML runner

In this case we want to observe turtles and marks.

In [4]:
runner = Similar2LogoWebRunner()
runner.config.setExportAgents(True)
runner.config.setExportMarks(True)
model = TurmiteSimulationModel(LogoSimulationParameters())
runner.initializeRunner(model)
runner.addProbe('Real time matcher', RealTimeMatcherProbe(20.0))
runner.showView()

No Outputs